In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Feb  6 16:08:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    49W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
import os
import torch
import numpy as np
from scipy.io import loadmat
from typing import Tuple,List,Callable
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
main_dir = '/content/drive/MyDrive/superresolution/nonlinear_classification'
data_dir = f'{main_dir}/data/features/SIM_by_crystal'
image_dir = f'{main_dir}/data/digit_image'

In [ ]:
sample_dir = f'{data_dir}/1f'
torch.save(model.state_dict(), filename)
filename = f'{sample_dir}/train_loss.pt'
torch.save(train_loss, filename)
filename = f'{sample_dir}/test_loss.pt'
torch.save(test_loss, filename)
filename = f'{sample_dir}/test_psnr.pt'
torch.save(test_psnr, filename)
filename = f'{sample_dir}/test_ssim.pt'
torch.save(test_ssim, filename)

In [ ]:
test_proportion = 0.2 # 20% of data for testing

def process_data(directory: str) -> None:
    # clean and split data into train and test sets
    raw = f'{directory}/raw'
    clean = f'{directory}/clean_split'
    for dirname in os.listdir(raw): #iterate through the files in directory
        image_paths = []
        digits = []
        raw_data = None
        dir = os.path.join(raw, dirname)
        # checking if it is a directory
        if os.path.isdir(dir):
            for filename in os.listdir(dir): #iterate through the files in directory
                file = os.path.join(dir, filename)
                # checking if it is a file
                if os.path.isfile(file):
                    data_dict = loadmat(file)
                    if raw_data is None:
                        raw_data = data_dict['conv_cp'].transpose()
                    else:
                        data = data_dict['conv_cp'].transpose()
                        raw_data = np.append(raw_data, data, axis=0)
                    image_nums = data_dict['ImageName'][0]
                    digit = data_dict['digit'][0][0] % 10
                    for i in range(len(image_nums)):
                        image_num = image_nums[i]
                        image_paths.append(f'{image_dir}/{digit}/image{str(image_num)}.png')
                        digits.append(digit)
                else:
                    print(file)
                    raise Exception("this is not a file")
            labels = []
            for i in range(len(digits)):
                label = (image_paths[i], digits[i])
                labels.append(label)
            X_train, X_test, y_train, y_test = train_test_split(raw_data, labels, test_size=test_proportion) # split training and testing data
            train = (X_train, y_train)
            test = (X_test, y_test)
            save = f'{clean}/{dirname}'
            torch.save(train, f'{save}_train.pt')
            torch.save(test, f'{save}_test.pt')
        else:
            raise Exception("this is not a directory")

In [ ]:
sample_dir = f'{data_dir}/Crystal40mm'
process_data(sample_dir)